In [2]:
import pandas as pd


data = pd.read_csv('dataset.csv')
data

/tmp/ipykernel_199001/1513548950.py:4: DtypeWarning: Columns (0,3,5,6,7,8,9,10,11,24) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('dataset.csv')


,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41.0,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.011700,0.0887,0.566,97.091,235440.0,tl
1,004s3t0ONYlzxII9PLgU6z,i feel alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28.0,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.009940,0.3470,0.404,135.225,373512.0,en
2,00chLpzhgVjxs1zKC9UScL,poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0.0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.007230,0.4890,0.650,111.904,262467.0,en
3,00cqd6ZsSkLZqGMlQCR0Zo,baby it's cold outside (feat. christina aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41.0,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.000000,0.0664,0.405,118.593,243067.0,en
4,00emjlCv9azBN0fzuuyLqy,dumb litty,KARD,Get up out of my business You don't keep me fr...,65.0,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.000000,0.1380,0.240,130.018,193160.0,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41064,NaN,find home,Green-House,NaN,35.0,NaN,NaN,NaN,NaN,NaN,...,-17.100,0,0.0413,0.99300,0.824000,0.0984,0.177,171.587,193450.0,NaN
41065,NaN,all gone,Micatone,NaN,27.0,NaN,NaN,NaN,NaN,NaN,...,-10.174,0,0.0329,0.85800,0.000016,0.0705,0.335,73.016,257067.0,NaN
41066,NaN,peine perdue,Smash Hit Combo,NaN,34.0,NaN,NaN,NaN,NaN,NaN,...,-4.683,0,0.0712,0.00003,0.000136,0.6660,0.262,105.000,216222.0,NaN
41067,NaN,salomon's gate,Beherit,NaN,29.0,NaN,NaN,NaN,NaN,NaN,...,-12.757,0,0.1340,0.00129,0.916000,0.2560,0.355,131.363,219693.0,NaN
